<a href="https://colab.research.google.com/github/veroorli/ProjetProg/blob/master/DM2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Querying nested data 

### examples for the official doc





### HOUSING DATASET

####1)Traitement de données: 

1)On sélectionne les colonnes utiles pour le ML 

2)Pour garantir que les valeurs soient du bon type on cast les valeurs (élimine les lignes qui dont les valeurs ne sont pas du bon type)

3)On enlève les enregistrements qui ont des valeurs nulles

In [ ]:
data=data.select(col("id").cast("long"),col("type").cast("string"),col("sqfeet").cast("float"),col("price").cast("float"),col("long").cast("float"),col("lat").cast("float"))
data.show()
data = data.na.drop("any")#drop les valeurs nulles


In [ ]:
sample =data.sample(0.01).persist()
sample.count()

In [ ]:
sample.show()


#### 2)Analyzers

#####longitude
On analyse la colonne longitude :complète (sans valeur manquante), 25% des locations sont sur la même longitude qu'une autre location, 


In [ ]:
# set these variables
_data = sample
_colName = "long"

#reuse from tutorial
from pydeequ.analyzers import *

analysisResult = AnalysisRunner(spark) \
                    .onData(_data) \
                    .addAnalyzer(Size()) \
                    .addAnalyzer(Completeness(_colName)) \
                    .addAnalyzer(Distinctness(_colName)) \
                    .run()
analysisResult_df = AnalyzerContext.successMetricsAsDataFrame(spark, analysisResult)
analysisResult_df.show()

+-------+--------+------------+------------------+
| entity|instance|        name|             value|
+-------+--------+------------+------------------+
|Dataset|       *|        Size|            3663.0|
| Column|    long|Completeness|               1.0|
| Column|    long|Distinctness|0.7570297570297571|
+-------+--------+------------+------------------+



#####surface
On s'intéresse aux corrélations du prix avec la surface corrélation de 33%

In [ ]:
# set these variables
_data = sample
_colName = "price"

#reuse from tutorial
from pydeequ.analyzers import *

analysisResult = AnalysisRunner(spark) \
                    .onData(_data) \
                    .addAnalyzer(Size()) \
                    .addAnalyzer(Completeness(_colName)) \
                    .addAnalyzer(Correlation("price", "sqfeet"))\
                    .run()
analysisResult_df = AnalyzerContext.successMetricsAsDataFrame(spark, analysisResult)
analysisResult_df.show()

+-----------+------------+------------+------------------+
|     entity|    instance|        name|             value|
+-----------+------------+------------+------------------+
|    Dataset|           *|        Size|            3663.0|
|     Column|       price|Completeness|               1.0|
|Mutlicolumn|price,sqfeet| Correlation|0.3305742906358196|
+-----------+------------+------------+------------------+



#### 3)profile

In [ ]:
for col, profile in result.profiles.items():
       print(profile)

#### 4) Suggestions

In [ ]:
from pydeequ.suggestions import *

suggestionResult = ConstraintSuggestionRunner(spark) \
             .onData(_data) \
             .addConstraintRule(DEFAULT()) \
             .run()

# Constraint Suggestions in JSON format
print(json.dumps(suggestionResult,indent=4))


#####Conclusion des suggestions pour la suite: 

Pour la surface sqfeet et la colonne price on peut vérifier qu'il n'y a pas de valeur négative ou nulle.

Pour la colonne id on peut vérifier qu'il y a bien un id par location pas de doublon.

Pour la colonne type, on peut vérifier que les valeurs catégoriques sont "apartment", "house", "townhouse", "condo","duplex","manufactured","flat","cottage/cabin", "loft", "in-law".
        
Il est aussi suggéré de vérifier que la latitude n'est pas négative . 



#### 5) constraint verification

In [ ]:
from pydeequ.checks import *
from pydeequ.verification import *

check = Check(spark, CheckLevel.Warning, "Review Check")

checkResult = VerificationSuite(spark) \
    .onData(_data) \
    .addCheck(
        check
        .isComplete("id")  \
        .isUnique("id")  \
        .isNonNegative("id")\
        .isNonNegative("price")\
        .isComplete("price")  \
        .isNonNegative("sqfeet")\
        .isComplete("sqfeet")  \
        .isContainedIn("type", ["apartment", "house", "townhouse", "condo","duplex","manufactured","flat","cottage/cabin", "loft", "in-law"])\
        .isNonNegative("lat"))  \
    .run()

checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show()

### Suggested data sources 
 
https://github.com/sdf94/deequ-examples/tree/master/data

https://www.kaggle.com/austinreese/craigslist-carstrucks-data

https://www.kaggle.com/austinreese/usa-housing-listings

http://insideairbnb.com/get-the-data.html
